In [2]:
! pip install dotenv

  Using cached python_dotenv-1.1.1-py3-none-any.whl.metadata (24 kB)
Using cached python_dotenv-1.1.1-py3-none-any.whl (20 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [dotenv]


In [3]:
# recent_trades_inspect.py
import os
import requests
from datetime import datetime, timezone
from dotenv import load_dotenv
import polars as pl

BASE = "https://api.binance.com"

def iso(ms: int) -> str:
    return datetime.fromtimestamp(ms/1000, tz=timezone.utc).isoformat()

def fetch_recent_trades(symbol: str, limit: int = 1000):
    """
    /api/v3/trades (Recent Trades List)
    - Public, KHÔNG cần API key/signature
    - Trả về các trường: id, price, qty, quoteQty, time, isBuyerMaker, isBestMatch
    - Lấy tối đa 'limit' trade gần nhất (không phân trang lùi theo thời gian được)
    """
    url = f"{BASE}/api/v3/trades"
    params = {"symbol": symbol.upper(), "limit": limit}
    r = requests.get(url, params=params, timeout=15)
    r.raise_for_status()
    return r.json()

def summarize_recent_trades(trades: list[dict]) -> pl.DataFrame:
    """
    Tạo bảng tóm tắt nhỏ: đếm/khối lượng phía BUY-taker vs SELL-taker
    """
    if not trades:
        return pl.DataFrame({
            "side": pl.Series([], dtype=pl.Utf8),
            "count": pl.Series([], dtype=pl.Int64),
            "base_sum": pl.Series([], dtype=pl.Float64),
            "quote_sum": pl.Series([], dtype=pl.Float64),
            "avg_price": pl.Series([], dtype=pl.Float64),
            "avg_size": pl.Series([], dtype=pl.Float64),
        })

    df = pl.DataFrame({
        "time":     [t["time"] for t in trades],           # ms
        "price":    [float(t["price"]) for t in trades],
        "qty":      [float(t["qty"]) for t in trades],
        "quoteQty": [float(t.get("quoteQty", 0.0)) for t in trades],
        "isBuyerMaker": [bool(t["isBuyerMaker"]) for t in trades],
        "isBestMatch": [bool(t.get("isBestMatch", False)) for t in trades],
    }).with_columns([
        # m=False -> buyer is taker -> TAKER BUY
        # m=True  -> buyer is maker -> TAKER SELL
        pl.when(pl.col("isBuyerMaker") == False)
          .then(pl.lit("TAKER_BUY")).otherwise(pl.lit("TAKER_SELL")).alias("side"),
        (pl.col("price") * pl.col("qty")).alias("quote"),
        pl.from_epoch(pl.col("time")/1000, time_unit="s").alias("ts"),
    ])

    summary = (df.group_by("side")
                 .agg([
                     pl.count().alias("count"),
                     pl.col("qty").sum().alias("base_sum"),
                     pl.col("quote").sum().alias("quote_sum"),
                     pl.col("price").mean().alias("avg_price"),
                     pl.col("qty").mean().alias("avg_size"),
                 ])
                 .sort("side"))
    return summary




In [4]:

# Load .env (không cần cho endpoint này, nhưng giữ thói quen nhất quán)
load_dotenv()
API_KEY = os.getenv("BINANCE_API_KEY")
SECRET  = os.getenv("BINANCE_SECRET_KEY")

# Bạn sửa danh sách symbol ở đây
SYMBOLS = ["BNBUSDT", "BTCUSDT"]
LIMIT   = 1000  # tối đa 1000

for sym in SYMBOLS:
    trades = fetch_recent_trades(sym, limit=LIMIT)
    print(f"\n=== {sym} | recentTrades: {len(trades)} rows ===")

    # In 5 dòng raw để kiểm tra cấu trúc
    for t in trades[:5]:
        print(f"id={t['id']} time={iso(t['time'])} price={t['price']} qty={t['qty']} "
                f"quoteQty={t.get('quoteQty')} isBuyerMaker={t['isBuyerMaker']} isBestMatch={t.get('isBestMatch')}")

    # Tóm tắt BUY/SELL taker
    summary = summarize_recent_trades(trades)
    print("\n-- Summary by taker side --")
    print(summary)


=== BNBUSDT | recentTrades: 1000 rows ===
id=1174731265 time=2025-09-25T11:25:28.713000+00:00 price=983.82000000 qty=0.00600000 quoteQty=5.90292000 isBuyerMaker=True isBestMatch=True
id=1174731266 time=2025-09-25T11:25:28.713000+00:00 price=983.82000000 qty=0.00600000 quoteQty=5.90292000 isBuyerMaker=True isBestMatch=True
id=1174731267 time=2025-09-25T11:25:28.713000+00:00 price=983.82000000 qty=0.13200000 quoteQty=129.86424000 isBuyerMaker=True isBestMatch=True
id=1174731268 time=2025-09-25T11:25:28.734000+00:00 price=983.83000000 qty=0.01700000 quoteQty=16.72511000 isBuyerMaker=False isBestMatch=True
id=1174731269 time=2025-09-25T11:25:28.945000+00:00 price=983.83000000 qty=0.05500000 quoteQty=54.11065000 isBuyerMaker=False isBestMatch=True

-- Summary by taker side --
shape: (2, 6)
┌────────────┬───────┬──────────┬──────────────┬────────────┬──────────┐
│ side       ┆ count ┆ base_sum ┆ quote_sum    ┆ avg_price  ┆ avg_size │
│ ---        ┆ ---   ┆ ---      ┆ ---          ┆ ---     

/tmp/ipykernel_224208/757862269.py:58: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
(Deprecated in version 0.20.5)
  pl.count().alias("count"),



=== BTCUSDT | recentTrades: 1000 rows ===
id=5256097039 time=2025-09-25T11:25:32.197000+00:00 price=111496.21000000 qty=0.00005000 quoteQty=5.57481050 isBuyerMaker=True isBestMatch=True
id=5256097040 time=2025-09-25T11:25:32.197000+00:00 price=111496.21000000 qty=0.00005000 quoteQty=5.57481050 isBuyerMaker=True isBestMatch=True
id=5256097041 time=2025-09-25T11:25:32.197000+00:00 price=111496.21000000 qty=0.00005000 quoteQty=5.57481050 isBuyerMaker=True isBestMatch=True
id=5256097042 time=2025-09-25T11:25:32.197000+00:00 price=111496.21000000 qty=0.00005000 quoteQty=5.57481050 isBuyerMaker=True isBestMatch=True
id=5256097043 time=2025-09-25T11:25:32.197000+00:00 price=111496.21000000 qty=0.00005000 quoteQty=5.57481050 isBuyerMaker=True isBestMatch=True

-- Summary by taker side --
shape: (2, 6)
┌────────────┬───────┬──────────┬───────────────┬───────────────┬──────────┐
│ side       ┆ count ┆ base_sum ┆ quote_sum     ┆ avg_price     ┆ avg_size │
│ ---        ┆ ---   ┆ ---      ┆ ---   